In [ ]:
TANGO_HOST="10.164.10.47:10000"
correlator_setup = 'two_receptor'
visibility_ip = "10.165.21.2"
dish_ids = ["SKA001"]
correlator_setup = "1fsp"

In [ ]:
from PyTango import DeviceProxy, AttributeProxy
import os
import json
from time import sleep
from datetime import datetime

os.environ["TANGO_HOST"] = TANGO_HOST

csp_server = "mid-csp/subarray/01"
csp_dp = DeviceProxy(csp_server)

with open(f'../../data/mid_telescope/csplmc/assign_resources.json', 'r') as json_data:
    d = json.load(json_data)
    d["dish"]["receptor_ids"] = dish_ids
    print(d)
    csp_dp.AssignResources(json.dumps(d))

sleep(3)

print(csp_dp.obsState)

with open(f'../../data/mid_telescope/csplmc/configure_scan.json', 'r') as json_data:
    d = json.load(json_data)
    d["cbf"]["fsp"][0]["output_host"][0][1] = visibility_ip
    print(d)
    csp_dp.configure(json.dumps(d))

sleep(3)
print(csp_dp.obsState)
sleep(5)
print(csp_dp.obsState)

In [ ]:
print(csp_dp.obsState)

In [ ]:
now = datetime.now()

with open(f'../../data/mid_telescope/csplmc/delay_model.json', 'r') as json_data:
    delay_model = json.load(json_data)

delay_model["epoch"] = float(int(now.timestamp()))

print(delay_model)

tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

sleep(5)

with open(f'../../data/scan.json', 'r') as json_data:
    d = json.load(json_data)
    csp_dp.scan(json.dumps(d))

sleep(2)
print(csp_dp.obsState)

In [ ]:
csp_dp.EndScan()

In [ ]:
csp_dp.GoToIdle()
sleep(2)
csp_dp.ReleaseAllResources()